In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import os

In [18]:
os.getcwd()

'/media/dell/cdcc2551-b430-4b85-a0f1-c628cb818672/zhenjun/shamen/ag'

In [5]:
df = TabularDataset('./suyuan/training.csv')

In [6]:
label_column = 'origin' # specifies which column do we want to predict
savedir = './suyuan/test1/' # where to save trained models
predictor = TabularPredictor(
                        label=label_column,
                        path=savedir
                        ).fit(df,presets='best_quality',auto_stack="T")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #146~16.04.1-Ubuntu SMP Tue Apr 13 09:27:15 UTC 2021
CPU Count:          32
Memory Avail:       85.32 GB / 93.06 GB (91.7%)
Disk Space Avail:   5560.69 GB / 18460.53 GB (30.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=T): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect st

In [7]:
lboard = predictor.leaderboard()
lboard.sort_values(by='score_val', ascending=False)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_r9_BAG_L2,0.916468,accuracy,57.695629,1622.848833,3.077413,106.587636,2,True,55
1,WeightedEnsemble_L3,0.916468,accuracy,57.696804,1623.273118,0.001175,0.424285,3,True,63
2,CatBoost_r137_BAG_L2,0.916468,accuracy,57.725595,1542.303318,3.107380,26.042122,2,True,60
3,XGBoost_r89_BAG_L1,0.915672,accuracy,0.192994,12.222773,0.192994,12.222773,1,True,27
4,WeightedEnsemble_L2,0.915672,accuracy,0.193818,12.546500,0.000824,0.323727,2,True,39
...,...,...,...,...,...,...,...,...,...,...
58,NeuralNetTorch_r79_BAG_L1,0.898966,accuracy,0.311566,14.759942,0.311566,14.759942,1,True,13
59,NeuralNetTorch_r14_BAG_L1,0.898170,accuracy,0.237806,15.058985,0.237806,15.058985,1,True,37
60,NeuralNetFastAI_r191_BAG_L1,0.896579,accuracy,0.433364,25.367914,0.433364,25.367914,1,True,15
61,NeuralNetFastAI_r103_BAG_L1,0.891806,accuracy,0.360858,19.406666,0.360858,19.406666,1,True,36


In [8]:

test_data = df
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(columns=[label_column])  # delete label column to prove we're not cheating
test_data_nolab.head()
 
#预测->结果简明
#predictor = TabularPredictor.load("/media/dell/cdcc2551-b430-4b85-a0f1-c628cb818672/zhenjun/shamen/ag/test3")  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Predictions:  
 0        Bovine
1           Pig
2           Pig
3       Chicken
4           Pig
         ...   
1252        Pig
1253        Pig
1254        Pig
1255        Pig
1256        Pig
Name: origin, Length: 1257, dtype: object


In [12]:
test_data = TabularDataset('./suyuan/testing.csv')
perf = predictor.evaluate(test_data, auxiliary_metrics=False)
perf

Loaded data from: ./suyuan/testing.csv | Columns = 6604 / 6604 | Rows = 252 -> 252


In [17]:
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(columns=[label_column])  # delete label column to prove we're not cheating
test_data_nolab.head()
 
#预测->结果简明
#predictor = TabularPredictor.load("/media/dell/cdcc2551-b430-4b85-a0f1-c628cb818672/zhenjun/shamen/ag/test3")  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab,as_pandas=True)
y_pred.to_csv("test-result.csv")
#print("Predictions:  \n", y_pred)
#perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


In [15]:
preict_data_full = TabularDataset('./suyuan/prediction-data.csv')
preict_data_full
pred_probablities = predictor.predict(preict_data_full, as_pandas=True)
pred_probablities.to_csv('results_predict_50.csv', index=False, encoding='utf-8')
pred_probablities

Loaded data from: ./suyuan/prediction-data.csv | Columns = 6604 / 6604 | Rows = 101 -> 101


0      Chicken
1       Bovine
2      Chicken
3      Chicken
4       Bovine
        ...   
96     Chicken
97     Chicken
98     Chicken
99     Chicken
100     Bovine
Name: origin, Length: 101, dtype: object

In [16]:
pred_probablities.to_csv('./suyuan/results_predict_50.csv', index=False, encoding='utf-8')

In [20]:
os.getcwd()

'/media/dell/cdcc2551-b430-4b85-a0f1-c628cb818672/zhenjun/shamen/ag'